# Comparing results

> Follow instruction [here](https://towardsdatascience.com/extracting-song-data-from-the-spotify-api-using-python-b1e79388d50) for getting your own Spotify Developer API credentials

In [ ]:
# !pip install spotipy --user

In [1]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
PROJECT_NUM = !gcloud projects list --filter="$PROJECT_ID" --format="value(PROJECT_NUMBER)"
PROJECT_NUM = PROJECT_NUM[0]
LOCATION = 'us-central1'

print(f"PROJECT_ID: {PROJECT_ID}")
print(f"PROJECT_NUM: {PROJECT_NUM}")
print(f"LOCATION: {LOCATION}")

PROJECT_ID: hybrid-vertex
PROJECT_NUM: 934903580331
LOCATION: us-central1


In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
import re
from tqdm import tqdm

import pandas as pd
import json
from io import BytesIO
from pprint import pprint
import os

from google.cloud import storage

pd.set_option('display.max_columns', 100)

### Setup clients

In [7]:
# CURRENT_PATH = !pwd

# os.chdir(f'{CURRENT_PATH}')
# os.getcwd()

* If you registered credentials with the Spotify Web Developer API, easily query your playlists to create your own test examples
* see [spotipy docs](https://spotipy.readthedocs.io/en/2.19.0/#welcome-to-spotipy), [github examples](https://github.com/plamere/spotipy/tree/master/examples), and [source code](https://github.com/plamere/spotipy/blob/master/spotipy/client.py#L20)

In [6]:
import spotipy_secret_creds as creds

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['SPOTIPY_CLIENT_ID'] = creds.SPOTIPY_CLIENT_ID
os.environ['SPOTIPY_CLIENT_SECRET'] = creds.SPOTIPY_CLIENT_SECRET
os.environ['SPOTIFY_USERNAME'] = creds.SPOTIFY_USERNAME

SPOTIPY_CLIENT_ID=creds.SPOTIPY_CLIENT_ID
SPOTIPY_CLIENT_SECRET=creds.SPOTIPY_CLIENT_SECRET
SPOTIFY_USERNAME=creds.SPOTIFY_USERNAME

MAX_SEED_LENGTH = 5

In [8]:
# Authenticate
client_credentials_manager = SpotifyClientCredentials(
    client_id=creds.SPOTIPY_CLIENT_ID, 
    client_secret=creds.SPOTIPY_CLIENT_SECRET
)

sp = spotipy.Spotify(
    client_credentials_manager = client_credentials_manager,

)

# Create Test Instances

## (1) Use your own Spotify playlists

Keep in mind:
* it's possible your playlists have tracks that are not present in the Million Playlists Dataset
* That's OK - we want the model to generalize to unseen data!
* Let's see what the model associates them with...

### get user playlists

> To connect to you Spotify account, see [Client Credentials Flow](https://spotipy.readthedocs.io/en/2.19.0/#client-credentials-flow) for authentication

In [12]:
playlists = sp.user_playlists(f'{SPOTIFY_USERNAME}')
while playlists:
    for i, playlist in enumerate(playlists['items']):
        print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None

### Test playlist link

In [13]:
# muscale shoals
playlist_link = 'https://open.spotify.com/playlist/3GX5FLE0IxHNZtLye0ETgb?si=f99fa67315f14bbe'

playlist_URI = playlist_link.split("/")[-1].split("?")[0]

print(f"playlist_URI: {playlist_URI}")

track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

track_uris

playlist_URI: 3GX5FLE0IxHNZtLye0ETgb


['spotify:track:7hqesNgWCx8NZTHl4MXkPF',
 'spotify:track:0y2y5zcZr0Ciei8bf41cSS',
 'spotify:track:1Hi6XYNirfRidjPo18UUq0',
 'spotify:track:5U51nS2N1TDL1HEHEio7mC',
 'spotify:track:4euYSSvpQZSAQ4gThyL2QD',
 'spotify:track:5CahDIpRZSLStfqNdznMoE',
 'spotify:track:6l6ObuVjdZbc900Q2WgO7K',
 'spotify:track:3i5m63j2chfkDu3qmcb9m3',
 'spotify:track:1MMp1H2Kib2BCDtdL5nL63',
 'spotify:track:2l5kuq6JLqCA6Afx8gGh8V',
 'spotify:track:1Zxu9r1D4fKGaxBUxmIsWG',
 'spotify:track:5n9Ul19Pb8MROfm8eTI7UH',
 'spotify:track:1JFdocYRMthec2BNrkOF1b']

## Loop multiple playlists

### Helper functions

In [77]:
# def get_track_features(track_uri, count, playlist_uri, n_songs_pl):
def get_track_features(track_uri, count, playlist_uri):
    
    feature_dict = {}
    
    playlist_features = sp.playlist(playlist_uri)
    feature_dict['pl_name'] = playlist_features['name']
    feature_dict['description_pl'] = playlist_features['description']
    feature_dict['collaborative'] = str(playlist_features['collaborative'])
    # feature_dict['n_songs_pl'] = n_songs_pl
    
    track_meta = sp.track(track_uri)
    # capture track metadata
    feature_dict['track_pos'] = count
    feature_dict['track_uri'] = track_uri
    feature_dict['track_name'] = track_meta['name']
    feature_dict['duration_ms'] = float(track_meta['duration_ms'])
    feature_dict['track_pop'] = float(track_meta['popularity'])
    feature_dict['album_name'] = track_meta['album']['name']
    feature_dict['album_uri'] = track_meta['album']['uri']
    feature_dict['album_release_date'] = track_meta['album']['release_date']
    feature_dict['artist_name'] = track_meta['album']['artists'][0]['name']
    feature_dict['artist_uri'] = track_meta['album']['artists'][0]['uri']

    artist_meta = sp.artist(feature_dict['artist_uri'])
    # capture artist metadata
    feature_dict['artist_followers'] = float(artist_meta['followers']['total'])
    feature_dict['artist_popularity'] = float(artist_meta['popularity'])
    # artist_genres = artist_meta['genres']
    
    if artist_meta['genres']:
        feature_dict['artist_genres'] = " ".join([re.sub(' ','_',i) for i in artist_meta['genres']])
    else:
        feature_dict['artist_genres'] = "unknown"
    
    track_features = sp.audio_features(track_uri)[0]
    # capture track audio features
    feature_dict['duration'] = float(track_features['duration_ms'])
    feature_dict['acousticness'] = track_features['acousticness']
    feature_dict['danceability'] = track_features['danceability']
    feature_dict['energy'] = track_features['energy']
    feature_dict['instrumentalness'] = track_features['instrumentalness']
    feature_dict['key'] = track_features['key']
    feature_dict['liveness'] = track_features['liveness']
    feature_dict['loudness']= track_features['loudness']
    feature_dict['mode'] = track_features['mode']
    feature_dict['speechiness'] = track_features['speechiness']
    feature_dict['tempo'] = track_features['tempo']
    feature_dict['time_signature'] = track_features['time_signature']
    feature_dict['valence'] = track_features['valence']
    
    # TODO: print artist names for comparison later
    
    return feature_dict

def get_playlist_queries(playlist_uri):
    
    track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_uri)["items"]]
    
    # n_songs_pl = len(track_uris)
    playlist_featutre_list = []
    
    for count, track_uri in enumerate(track_uris):
        # results = get_track_features(track_uri, count, playlist_uri, n_songs_pl)
        results = get_track_features(track_uri, count, playlist_uri)
        playlist_featutre_list.append(results)
    
    return playlist_featutre_list

### Example Playlists

In [78]:
playlist_uris = []

playlist_links = [
    'https://open.spotify.com/playlist/3GX5FLE0IxHNZtLye0ETgb?si=80db2142bb684516', # muscle shoals
    'https://open.spotify.com/playlist/0XPJ39OCBhOw5OZa7udYYP?si=92551fefe7c64eb1', # disco
    'https://open.spotify.com/playlist/6imD2IJOyw3MEKdZ4XZqZ4?si=b29779098622481f', # space is the place
    # 'https://open.spotify.com/playlist/1E1EwxJyzjt6SYyfnp9mE8?si=0f1cda3332c14290', # all panic
    'https://open.spotify.com/playlist/1pGfqRD9CzyO9lOn9Fp09V?si=3f6c8067491a4105', # live panic - small
    'https://open.spotify.com/playlist/3HeHZi8VGEm6ZNHZ2FVRr6?si=ca0b107a42e84067', # biebs weeknd
]

for link in playlist_links:
    playlist_uri = link.split("/")[-1].split("?")[0]
    playlist_uris.append(playlist_uri)

print(f"playlist_uris: {playlist_uris}")

playlist_uris: ['3GX5FLE0IxHNZtLye0ETgb', '0XPJ39OCBhOw5OZa7udYYP', '6imD2IJOyw3MEKdZ4XZqZ4', '1pGfqRD9CzyO9lOn9Fp09V', '3HeHZi8VGEm6ZNHZ2FVRr6']


In [79]:
featureLIST = [] 

for uri in playlist_uris:
    results = get_playlist_queries(uri)
    featureLIST.append(results)

len(featureLIST)

5

#### Inspect last `N` songs of playlist

In [39]:
featureLIST[3][-MAX_SEED_LENGTH:]

In [20]:
from itertools import chain

test_df = pd.DataFrame(list(chain.from_iterable(featureLIST)))

print(test_df.shape)
test_df.head()

(65, 29)


,pl_name,description_pl,collaborative,track_pos,track_uri,track_name,duration_ms,track_pop,album_name,album_uri,album_release_date,artist_name,artist_uri,artist_followers,artist_popularity,artist_genres,duration,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Muscle Shoals,"a blend of country, gospel and R &amp; B, alab...",False,0,spotify:track:7hqesNgWCx8NZTHl4MXkPF,"Mary, Don't You Weep - Live at New Temple Miss...",446506,36,Amazing Grace (Live at New Temple Missionary B...,spotify:album:5pIUimaQ6XePPXbMUi3te0,1972,Aretha Franklin,spotify:artist:7nwUJBm0HE4ZxD3f5cy5ok,2977459,69,classic_soul jazz_blues memphis_soul soul sout...,446507,0.74400,0.572,0.270,0.000185,1,0.8580,-16.201,1,0.0378,81.720,3,0.610
1,Muscle Shoals,"a blend of country, gospel and R &amp; B, alab...",False,1,spotify:track:0y2y5zcZr0Ciei8bf41cSS,Confessions,324760,45,Carolina Confessions,spotify:album:6RVN0MPUbYBiYm2Otf6Li2,2018-10-05,Marcus King,spotify:artist:0FeWKiZSwBRdGzqeCdlH1a,130831,56,modern_blues_rock,324760,0.26900,0.357,0.437,0.000403,5,0.1510,-9.072,0,0.0312,135.730,3,0.447
2,Muscle Shoals,"a blend of country, gospel and R &amp; B, alab...",False,2,spotify:track:1Hi6XYNirfRidjPo18UUq0,Welcome ’Round Here,261546,49,Carolina Confessions,spotify:album:6RVN0MPUbYBiYm2Otf6Li2,2018-10-05,Marcus King,spotify:artist:0FeWKiZSwBRdGzqeCdlH1a,130831,56,modern_blues_rock,261547,0.00904,0.297,0.734,0.004430,0,0.0959,-7.255,1,0.0443,93.259,4,0.252
3,Muscle Shoals,"a blend of country, gospel and R &amp; B, alab...",False,3,spotify:track:5U51nS2N1TDL1HEHEio7mC,Down In the Flood,302306,45,Already Free,spotify:album:1I4Ydy7hVvINSdRHV7n4fS,2009-01-13,The Derek Trucks Band,spotify:artist:1YwfENKEZrowcmtR1nALZn,193670,44,blues blues_rock country_rock electric_blues j...,302307,0.18100,0.551,0.695,0.000154,0,0.1920,-7.111,1,0.0323,114.135,4,0.207
4,Muscle Shoals,"a blend of country, gospel and R &amp; B, alab...",False,4,spotify:track:4euYSSvpQZSAQ4gThyL2QD,Ain't Wastin' Time No More,221890,59,Eat A Peach,spotify:album:6D9I7GZsqc3pwDg3DFrtjs,1972-02-12,Allman Brothers Band,spotify:artist:4wQ3PyMz3WwJGI5uEqHUVR,1368075,63,album_rock blues_rock classic_rock country_roc...,221891,0.33700,0.453,0.607,0.023700,3,0.0998,-13.400,1,0.0345,91.468,4,0.724


## TODO
* `n_songs_pl`
* `num_artists_pl`
* `num_albums_pl`
* format ragged columns (<= 5)
* Add section for creating tet instances without Spotify account? e.g., favorite artists, tracks, and genres?

### Option 1

In [21]:
# test_df_2 = test_df.copy()

In [24]:
# test_df_2['n_songs_pl'] = test_df_2[test_df_2['pl_name'] == condition, :].apply(any_fn_you_want)
# feature_dict['pl_name'] = playlist_features['name'].str.strip().str.lower().replace('', '_')

# # dfd['new_B'] = dfd.groupby('A')['B'].transform('min')

# # test_df_2['n_songs_pl'] = test_df_2.groupby('track_uri')['pl_name'].count()
# test_df_2['n_songs_pl'] = test_df_2.groupby('pl_name')['ccc'].count()

unique_albums = test_df_2.groupby('pl_name')['album_uri'].nunique()
unique_artists = test_df_2.groupby('pl_name')['artist_uri'].nunique()
n_songs_pl = test_df_2.groupby('pl_name')['track_uri'].count()

print(f"unique_albums {unique_albums}\n")
print(f"unique_artists {unique_artists}\n")
print(f"n_songs_pl {n_songs_pl}")

pl_name
Disco                  5
Muscle Shoals         13
biebs weeknd          20
live panic - small    14
space is the place    13
Name: track_uri, dtype: int64

### Option 2

In [52]:
# featureLIST[3][-MAX_SEED_LENGTH + 1:]

print(f"Number of playlists {len(featureLIST)}")
print(f"Number of tracks in a single playlis {len(featureLIST[3])}")
print(f"Number of seedtracks during model training: {MAX_SEED_LENGTH}")

Number of playlists 5
Number of tracks in a single playlis 14
Number of seedtracks during model training: 5


In [120]:
# muscle_shoals_test = featureLIST[0][-MAX_SEED_LENGTH:]
# disco_test = featureLIST[1][-MAX_SEED_LENGTH:]
# space_place_test = featureLIST[2][-MAX_SEED_LENGTH:]
# panic_feat_test = featureLIST[3][-MAX_SEED_LENGTH:]
# biebs_wkd_test = featureLIST[4][-MAX_SEED_LENGTH:]

In [125]:
def get_test_instance(list_dict_test):
    '''
    create single test instances given a 
    list of dictionaries representing playlist tracks
    '''
    
    # model serving signature with candidate tower fields
    TEST_PL_QUERY = {
        'album_name_can': '',
        'album_name_pl': [ ], 
        'album_uri_can': '',
        'artist_followers_can': 0, 
        'artist_genres_can': "", 
        'artist_genres_pl': [ ], 
        'artist_name_can': '', 
        'artist_name_pl': [ ], 
        'artist_pop_can': 0, 
        'artist_pop_pl': [], 
        'artist_uri_can': '', 
        'artists_followers_pl': [ ], 
        'collaborative': '', 
        'description_pl': '', 
        'duration_ms_can': 0, 
        'duration_ms_songs_pl': [ ], 
        'n_songs_pl': 0, 
        'name': '', 
        'num_albums_pl': 0, 
        'num_artists_pl': 0, 
        'track_name_can': '', 
        'track_name_pl': [ ], 
        'track_pop_can': 0, 
        'track_pop_pl': [ ], 
        'track_uri_can': '', 
        'track_uri_pl': [ ],
    }

    counter = 0
    for track in list_dict_test:
        if counter == 0:
            TEST_PL_QUERY['collaborative'] = str(track['collaborative'])
            TEST_PL_QUERY['n_songs_pl'] = 14.0 # TODO: paramerterize
            TEST_PL_QUERY['num_albums_pl'] = 3.0 # TODO: paramerterize
            TEST_PL_QUERY['num_artists_pl'] = 2.0 # TODO: paramerterize
            TEST_PL_QUERY['name'] = track['pl_name']
            TEST_PL_QUERY['description_pl'] = track['description_pl']
        else:
            # do these
            TEST_PL_QUERY['album_name_pl'].append(track['album_name'])
            TEST_PL_QUERY['artist_genres_pl'].append(track['artist_genres'])
            TEST_PL_QUERY['artist_name_pl'].append(track['artist_name'])
            TEST_PL_QUERY['artist_pop_pl'].append(track['artist_popularity'])
            TEST_PL_QUERY['artists_followers_pl'].append(track['artist_followers'])
            # TEST_PL_QUERY['collaborative'].append(track['XXXXXX'])
            # TEST_PL_QUERY['description_pl'].append(track['XXXXXX'])
            TEST_PL_QUERY['duration_ms_songs_pl'].append(track['duration_ms'])
            # TEST_PL_QUERY['n_songs_pl'].append(track['XXXXXX'])
            # TEST_PL_QUERY['name'].append(track['XXXXXX'])
            # TEST_PL_QUERY['num_albums_pl'].append(track['XXXXXX'])
            # TEST_PL_QUERY['num_artists_pl'].append(track['XXXXXX'])
            TEST_PL_QUERY['track_name_pl'].append(track['track_name'])
            TEST_PL_QUERY['track_pop_pl'].append(track['track_pop'])
            TEST_PL_QUERY['track_uri_pl'].append(track['track_uri'])

        counter=+1
        
    return TEST_PL_QUERY

# Query Matching Engine

### TODO:
* parametrize this section
* structure section and notebook for readers

In [139]:
from google.cloud import aiplatform as vertex_ai
# from google.cloud import aiplatform_v1beta1 #needed for matching engine calls

# Vertex SDK 
vertex_ai.init(project=PROJECT_ID, location=LOCATION)

import tensorflow as tf
import tensorflow_recommenders as tfrs

from two_tower_src import two_tower as tt

import time

### Define VPC Network

In [41]:
VPC_NETWORK_NAME = 'ucaip-haystack-vpc-network' 

VPC_NETWORK_NAME_URI = f'projects/{PROJECT_NUM}/global/networks/{VPC_NETWORK_NAME}'

print("VPC_NETWORK_RESOURCE_URI:", VPC_NETWORK_NAME_URI)

VPC_NETWORK_RESOURCE_URI: projects/934903580331/global/networks/ucaip-haystack-vpc-network


### Define model endpoint

In [ ]:
### Deployed Model Endpoints

# !gcloud beta ai index-endpoints list --project={PROJECT_ID} --region={LOCATION}

In [42]:
query_endpoint_name = "Spotify Playist Model Endpoint" 

In [43]:
endpoints = vertex_ai.Endpoint.list(
    filter='display_name="{}"'.format(query_endpoint_name),
    order_by='create_time desc',
    project=PROJECT_ID, 
    location=LOCATION,
)

model_endpoint_uri = endpoints[0].resource_name

deployed_query_model_endpoint = vertex_ai.Endpoint(model_endpoint_uri)

print(f"query_endpoint_name: {query_endpoint_name}")
print(f"model_endpoint_uri: {model_endpoint_uri}\n")
print(f"deployed_query_model_endpoint: {deployed_query_model_endpoint}")

query_endpoint_name: Spotify Playist Model Endpoint
model_endpoint_uri: projects/934903580331/locations/us-central1/endpoints/1185154787486728192

deployed_query_model_endpoint: <google.cloud.aiplatform.models.Endpoint object at 0x7f115d0cfe10> 
resource name: projects/934903580331/locations/us-central1/endpoints/1185154787486728192


In [45]:
deployed_models = []

for model in deployed_query_model_endpoint.list_models():
    deployed_models.append(model)
    pprint(model)
    
if len(deployed_models) > 0:
    model = deployed_models[0]
    deployed_model_id = model.id
    deployed_model_uri = model.model
else:
    print("No deployed model ")

id: "1391950934438838272"
model: "projects/934903580331/locations/us-central1/models/8649686451899858944"
display_name: "Spotify Playlist Query Model"
create_time {
  seconds: 1665582682
  nanos: 65459000
}
dedicated_resources {
  machine_spec {
    machine_type: "n1-standard-4"
  }
  min_replica_count: 1
  max_replica_count: 2
}
model_version_id: "1"



### Define index endpoint

In [ ]:
# !gcloud beta ai index-endpoints list --project={PROJECT_ID} --region={LOCATION}

In [46]:
index_endpoint_display_name = "index_endpoint_for_demo"
deployed_index_id = "deployed_spotify_v1"
INDEX_ENDPOINT_ID = '259220861364469760'

In [47]:
index_endpoint_uri = f'projects/{PROJECT_NUM}/locations/{LOCATION}/indexEndpoints/{INDEX_ENDPOINT_ID}'

# initialize endpoint object
deployed_index_endpoint = vertex_ai.MatchingEngineIndexEndpoint(index_endpoint_uri)
deployed_index_endpoint

resource name: projects/934903580331/locations/us-central1/indexEndpoints/259220861364469760

In [48]:
deployed_index_endpoint.deployed_indexes

[id: "deployed_spotify_v1"
index: "projects/934903580331/locations/us-central1/indexes/7391796771212492800"
display_name: "deployed_spotify_v1"
create_time {
  seconds: 1665582418
  nanos: 81384000
}
private_endpoints {
  match_grpc_address: "10.41.2.5"
}
index_sync_time {
  seconds: 1665593758
  nanos: 876025000
}
automatic_resources {
  min_replica_count: 2
  max_replica_count: 2
}
deployment_group: "default"
]

## Convert Test Instance embedding vector(s)

In [136]:
# MODEL_PATH = 'gs://two-tower-models'
# QUERY_MODEL_DIR = f'{MODEL_PATH}/query_model'
# loaded_query_model = tf.saved_model.load(QUERY_MODEL_DIR)
# loaded_query_model.signatures

# Tests
# muscle_shoals_test = featureLIST[0][-MAX_SEED_LENGTH:]

# TEST_QUERY_mshoals = get_test_instance(muscle_shoals_test)

# pprint(TEST_QUERY_mshoals)

In [127]:
# playlist_emb = deployed_query_model_endpoint.predict([TEST_QUERY_mshoals])

# print(f"Length of playlist embedding vector: {len(playlist_emb.predictions)}")

# playlist_emb

Length of playlist embedding vector: 1


Prediction(predictions=[[0.00448607281, 0.0799401, 0.150346503, 0.16323255, -0.0965869948, -0.0266425926, 0.052257821, -0.124586716, 0.110679455, -0.00614864519, -0.0912500396, 0.0200050846, -0.0869326591, 0.138208762, 0.12699759, 0.204420358, -0.0709763244, -0.082770139, 0.0900019258, 0.0547221042, -0.0741926059, -0.12883544, -0.0481014289, 0.10176421, 0.0879789665, -0.0333082, -0.021751957, 0.0639551729, -0.0413546972, 0.0909351856, -0.00764867337, -0.00112683303, -0.0371141247, 0.082656309, 0.0489563122, 0.0472388677, -0.0138592515, -0.0311043, 0.168435723, 0.08697097, 0.120492741, 0.0922107548, 0.0446227379, -0.07278575, 0.0353432596, 0.0687891692, -0.040090017, -0.0239148494, 0.0689484, 0.110485986, 0.0752607733, -0.0382980481, -0.0762647837, 0.0975420401, 0.0483518131, -0.149040759, -0.0422194563, -0.0380143598, 0.0473450534, -0.0395774953, 0.0801844522, -0.0919230878, -0.0736038685, 0.00854421128, 0.0187642556, -0.0902012661, -0.0119783813, -0.0657001063, -0.0394202769, -0.17201

## Retrieve nearest neighbors in deployed index

In [142]:
def candidate_retrieval(query_instance, num_neighbs=10):
    '''
    TODO: some stuff
    '''
    # here
    playlist_emb = deployed_query_model_endpoint.predict([query_instance])
    
    candidate_tracks = deployed_index_endpoint.match(
        deployed_index_id='deployed_spotify_v1',
        queries=playlist_emb.predictions,
        num_neighbors=10
    )
    # pprint(f"Retrieved candidate tracks: {candidate_tracks}")
    return candidate_tracks

In [143]:
def interpret_results(candidate_tracks):
    '''
    TODO:
    '''
    # here
    results = []

    for neighbors in candidate_tracks[0]:
        track_dict = {}
        track_index_id = str(neighbors.id).lstrip('b').replace("'","")
        print(track_index_id)
        track_meta = sp.track(track_index_id)
        track_dict['track_name'] = track_meta['name']
        track_dict['artist_name'] = track_meta['artists'][0]['name']
        track_dict['track_preview_url'] = track_meta['preview_url']
        track_dict['track_spotify_url'] = track_meta['external_urls']['spotify']
        track_dict['track_uri'] = track_index_id
        track_dict['neighbor_distance'] = neighbors.distance
        results.append(track_dict)

    return results

### Muscle Shoals Test

In [144]:
start_time = time.time()

candidates_muscle_shoals = candidate_retrieval(TEST_QUERY_mshoals)
results_muscle_shoals = interpret_results(candidates_muscle_shoals)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed Time: {round(elapsed_time, 2)}")

results_muscle_shoals

spotify:track:7eUDMQAudTPMUP766rZ90N
spotify:track:7sKoHCtHYMqUA7SBwZFGaq
spotify:track:1pfNFUzZIhwPx1m9eqOlfX
spotify:track:6sjsrOD8fgYg1K4oWrlXH3
spotify:track:6Qsk1VAuuOzJNmIJ9GtDk9
spotify:track:012GkdJdpax04j6llSlAVY
spotify:track:25kXhsMguNGnSVWPxsd34Z
spotify:track:0ikhKa5Su4Lq4luc2EQpY3
spotify:track:2ugxxlY8ukD5eINjz2Jvti
spotify:track:20GlNYgJviItWPirfz7kfx
Elapsed Time: 0.63


[{'track_name': 'After Hours',
  'artist_name': 'Babyshambles',
  'track_preview_url': 'https://p.scdn.co/mp3-preview/3192572c1ad42e7b7a6de3618d70847815c995c1?cid=2dce494e64a74be980138668f4402b97',
  'track_spotify_url': 'https://open.spotify.com/track/7eUDMQAudTPMUP766rZ90N',
  'track_uri': 'spotify:track:7eUDMQAudTPMUP766rZ90N',
  'neighbor_distance': 8.880292892456055},
 {'track_name': 'Joy Of Nothing',
  'artist_name': 'Foy Vance',
  'track_preview_url': None,
  'track_spotify_url': 'https://open.spotify.com/track/7sKoHCtHYMqUA7SBwZFGaq',
  'track_uri': 'spotify:track:7sKoHCtHYMqUA7SBwZFGaq',
  'neighbor_distance': 8.57666301727295},
 {'track_name': 'I Went to Your Wedding (Rerecorded)',
  'artist_name': 'Patti Page',
  'track_preview_url': None,
  'track_spotify_url': 'https://open.spotify.com/track/1pfNFUzZIhwPx1m9eqOlfX',
  'track_uri': 'spotify:track:1pfNFUzZIhwPx1m9eqOlfX',
  'neighbor_distance': 7.165894031524658},
 {'track_name': 'Lover',
  'artist_name': 'Stan Kenton',
  't

### Disco Test

In [147]:
start_time = time.time()

candidates_disco = candidate_retrieval(TEST_QUERY_disco)

results_disco = interpret_results(candidates_disco)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed Time: {round(elapsed_time, 2)}")

results_disco

spotify:track:4Cod4nd7e3lOH29SPKrYzl
spotify:track:6x5x8DhFw1hiHFKIeGAI2S
spotify:track:2BTxbbLJiPd1ySKtvsBgn8
spotify:track:25kXhsMguNGnSVWPxsd34Z
spotify:track:223oeM6IwwmmIurPkjJSmO
spotify:track:4v4dxYXh7O27D4giuuwNhI
spotify:track:2w4VFk9C9OvJmj7NqFmtLZ
spotify:track:2ugxxlY8ukD5eINjz2Jvti
spotify:track:7I29NUhmNYZgLkEq3nnp7W
spotify:track:4d1r0CeqWIgdXj2YbFqdr4
Elapsed Time: 0.55


[{'track_name': 'Objection Overruled',
  'artist_name': 'Don Carlos',
  'track_preview_url': None,
  'track_spotify_url': 'https://open.spotify.com/track/4Cod4nd7e3lOH29SPKrYzl',
  'track_uri': 'spotify:track:4Cod4nd7e3lOH29SPKrYzl',
  'neighbor_distance': 3.7719290256500244},
 {'track_name': 'Ready',
  'artist_name': 'Andre Saint-Albin',
  'track_preview_url': 'https://p.scdn.co/mp3-preview/af137b9cc09f4f5dbdddd9278ffdcd9d55e590ee?cid=2dce494e64a74be980138668f4402b97',
  'track_spotify_url': 'https://open.spotify.com/track/6x5x8DhFw1hiHFKIeGAI2S',
  'track_uri': 'spotify:track:6x5x8DhFw1hiHFKIeGAI2S',
  'neighbor_distance': 3.610707998275757},
 {'track_name': 'Gimme Three Steps - Live',
  'artist_name': 'Lynyrd Skynyrd',
  'track_preview_url': None,
  'track_spotify_url': 'https://open.spotify.com/track/2BTxbbLJiPd1ySKtvsBgn8',
  'track_uri': 'spotify:track:2BTxbbLJiPd1ySKtvsBgn8',
  'neighbor_distance': 3.569955825805664},
 {'track_name': 'Bella (Acoustic)',
  'artist_name': 'Jamie G

### Space is the Place Test

In [148]:
start_time = time.time()

candidates_space_place = candidate_retrieval(TEST_QUERY_space)

results_space_place = interpret_results(candidates_space_place)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed Time: {round(elapsed_time, 2)}")

results_space_place

spotify:track:2BTxbbLJiPd1ySKtvsBgn8
spotify:track:7sKoHCtHYMqUA7SBwZFGaq
spotify:track:14pce69FSX051ZFu1kcJrZ
spotify:track:5tLOTZ6xlaPo6RhvCpZYkV
spotify:track:1HY2o3VZgIVmJDo0xmH92L
spotify:track:7y4f0Zzto8hLBvuIwDhYkR
spotify:track:3Gq8CJtuJV1KlAfIJPpSDt
spotify:track:4Cod4nd7e3lOH29SPKrYzl
spotify:track:2w4VFk9C9OvJmj7NqFmtLZ
spotify:track:7MgkyepnUxzlvO4z65athy
Elapsed Time: 0.46


[{'track_name': 'Gimme Three Steps - Live',
  'artist_name': 'Lynyrd Skynyrd',
  'track_preview_url': None,
  'track_spotify_url': 'https://open.spotify.com/track/2BTxbbLJiPd1ySKtvsBgn8',
  'track_uri': 'spotify:track:2BTxbbLJiPd1ySKtvsBgn8',
  'neighbor_distance': 5.380218505859375},
 {'track_name': 'Joy Of Nothing',
  'artist_name': 'Foy Vance',
  'track_preview_url': None,
  'track_spotify_url': 'https://open.spotify.com/track/7sKoHCtHYMqUA7SBwZFGaq',
  'track_uri': 'spotify:track:7sKoHCtHYMqUA7SBwZFGaq',
  'neighbor_distance': 4.02401065826416},
 {'track_name': 'Where Do We Go',
  'artist_name': 'Immature',
  'track_preview_url': None,
  'track_spotify_url': 'https://open.spotify.com/track/14pce69FSX051ZFu1kcJrZ',
  'track_uri': 'spotify:track:14pce69FSX051ZFu1kcJrZ',
  'neighbor_distance': 3.6826207637786865},
 {'track_name': 'Creepin (feat. Freddie Gibbs)',
  'artist_name': 'Alex Wiley',
  'track_preview_url': None,
  'track_spotify_url': 'https://open.spotify.com/track/5tLOTZ6xl

### Biebs Test

In [149]:
start_time = time.time()

candidates_biebs = candidate_retrieval(TEST_QUERY_biebs)

results_biebs = interpret_results(candidates_biebs)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed Time: {round(elapsed_time, 2)}")

results_biebs

spotify:track:1wGtESc1dV3KhY2xN2zI91
spotify:track:2UONUN0rGMFtcOMJPWXHBp
spotify:track:234WGrviDDBJhpsf7KzQYE
spotify:track:3xvFTqHmlMqKjHgczCGn2C
spotify:track:42ueD8Ul16MPyS3Q1uKZKw
spotify:track:0d0iKb7tAJKpLZI5U6rb4W
spotify:track:1JK3vg465jyEpFqsyKCyFk
spotify:track:56wlUHi9IhwZzTjECaRHVd
spotify:track:3aKba03EaKaDAtaZq8tB4C
spotify:track:5hhk460lT2iG7pLjTDsdXf
Elapsed Time: 0.48


[{'track_name': 'What If',
  'artist_name': 'Story Untold',
  'track_preview_url': None,
  'track_spotify_url': 'https://open.spotify.com/track/1wGtESc1dV3KhY2xN2zI91',
  'track_uri': 'spotify:track:1wGtESc1dV3KhY2xN2zI91',
  'neighbor_distance': 8.3035249710083},
 {'track_name': 'Rabbit Hole',
  'artist_name': 'The Bunny The Bear',
  'track_preview_url': 'https://p.scdn.co/mp3-preview/9ac670df4100dddda77c1335b779a30d089e0213?cid=2dce494e64a74be980138668f4402b97',
  'track_spotify_url': 'https://open.spotify.com/track/2UONUN0rGMFtcOMJPWXHBp',
  'track_uri': 'spotify:track:2UONUN0rGMFtcOMJPWXHBp',
  'neighbor_distance': 7.805734157562256},
 {'track_name': 'El Cocinero',
  'artist_name': 'Dinastia Norteña',
  'track_preview_url': 'https://p.scdn.co/mp3-preview/afca30a3974fa9281bc9e1506a7a9b42faf3c7ed?cid=2dce494e64a74be980138668f4402b97',
  'track_spotify_url': 'https://open.spotify.com/track/234WGrviDDBJhpsf7KzQYE',
  'track_uri': 'spotify:track:234WGrviDDBJhpsf7KzQYE',
  'neighbor_dist

### Panic Test

In [150]:

start_time = time.time()

candidates_panic = candidate_retrieval(TEST_QUERY_panic)

results_panic = interpret_results(candidates_panic)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed Time: {round(elapsed_time, 2)}")

results_panic

spotify:track:7eUDMQAudTPMUP766rZ90N
spotify:track:7sKoHCtHYMqUA7SBwZFGaq
spotify:track:2BTxbbLJiPd1ySKtvsBgn8
spotify:track:25kXhsMguNGnSVWPxsd34Z
spotify:track:2ugxxlY8ukD5eINjz2Jvti
spotify:track:7MgkyepnUxzlvO4z65athy
spotify:track:5Qo2qbhJZEWE4Go6RNC4Ee
spotify:track:2KkqvaYudEQMQXRK8c8VTs
spotify:track:553ZNNYDXGnBR3HGceKWNQ
spotify:track:3Ij3BciihXLsMIlQIn0Ob2
Elapsed Time: 0.55


[{'track_name': 'After Hours',
  'artist_name': 'Babyshambles',
  'track_preview_url': 'https://p.scdn.co/mp3-preview/3192572c1ad42e7b7a6de3618d70847815c995c1?cid=2dce494e64a74be980138668f4402b97',
  'track_spotify_url': 'https://open.spotify.com/track/7eUDMQAudTPMUP766rZ90N',
  'track_uri': 'spotify:track:7eUDMQAudTPMUP766rZ90N',
  'neighbor_distance': 6.695947170257568},
 {'track_name': 'Joy Of Nothing',
  'artist_name': 'Foy Vance',
  'track_preview_url': None,
  'track_spotify_url': 'https://open.spotify.com/track/7sKoHCtHYMqUA7SBwZFGaq',
  'track_uri': 'spotify:track:7sKoHCtHYMqUA7SBwZFGaq',
  'neighbor_distance': 6.478202819824219},
 {'track_name': 'Gimme Three Steps - Live',
  'artist_name': 'Lynyrd Skynyrd',
  'track_preview_url': None,
  'track_spotify_url': 'https://open.spotify.com/track/2BTxbbLJiPd1ySKtvsBgn8',
  'track_uri': 'spotify:track:2BTxbbLJiPd1ySKtvsBgn8',
  'neighbor_distance': 5.356612205505371},
 {'track_name': 'Bella (Acoustic)',
  'artist_name': 'Jamie Grace',

## Archive

In [114]:
results = []

for neighbors in candidate_tracks[0]:
    track_dict = {}
    track_index_id = str(neighbors.id).lstrip('b').replace("'","")
    print(track_index_id)
    track_meta = sp.track(track_index_id)
    track_dict['track_name'] = track_meta['name']
    track_dict['artist_name'] = track_meta['artists'][0]['name']
    track_dict['track_preview_url'] = track_meta['preview_url']
    track_dict['track_spotify_url'] = track_meta['external_urls']['spotify']
    track_dict['track_uri'] = track_index_id
    track_dict['neighbor_distance'] = neighbors.distance
    results.append(track_dict)
    
results

spotify:track:7eUDMQAudTPMUP766rZ90N
spotify:track:7sKoHCtHYMqUA7SBwZFGaq
spotify:track:2BTxbbLJiPd1ySKtvsBgn8
spotify:track:25kXhsMguNGnSVWPxsd34Z
spotify:track:2ugxxlY8ukD5eINjz2Jvti
spotify:track:7MgkyepnUxzlvO4z65athy
spotify:track:5Qo2qbhJZEWE4Go6RNC4Ee
spotify:track:2KkqvaYudEQMQXRK8c8VTs
spotify:track:553ZNNYDXGnBR3HGceKWNQ
spotify:track:3Ij3BciihXLsMIlQIn0Ob2


[{'track_name': 'After Hours',
  'artist_name': 'Babyshambles',
  'track_preview_url': 'https://p.scdn.co/mp3-preview/3192572c1ad42e7b7a6de3618d70847815c995c1?cid=2dce494e64a74be980138668f4402b97',
  'track_spotify_url': 'https://open.spotify.com/track/7eUDMQAudTPMUP766rZ90N',
  'track_uri': 'spotify:track:7eUDMQAudTPMUP766rZ90N',
  'neighbor_distance': 6.695947170257568},
 {'track_name': 'Joy Of Nothing',
  'artist_name': 'Foy Vance',
  'track_preview_url': None,
  'track_spotify_url': 'https://open.spotify.com/track/7sKoHCtHYMqUA7SBwZFGaq',
  'track_uri': 'spotify:track:7sKoHCtHYMqUA7SBwZFGaq',
  'neighbor_distance': 6.478202819824219},
 {'track_name': 'Gimme Three Steps - Live',
  'artist_name': 'Lynyrd Skynyrd',
  'track_preview_url': None,
  'track_spotify_url': 'https://open.spotify.com/track/2BTxbbLJiPd1ySKtvsBgn8',
  'track_uri': 'spotify:track:2BTxbbLJiPd1ySKtvsBgn8',
  'neighbor_distance': 5.356612205505371},
 {'track_name': 'Bella (Acoustic)',
  'artist_name': 'Jamie Grace',

In [109]:
# track_test_ = 'spotify:track:7eUDMQAudTPMUP766rZ90N'
# track_test_ = 'spotify:track:7sKoHCtHYMqUA7SBwZFGaq'
# track_test_ = 'spotify:track:2BTxbbLJiPd1ySKtvsBgn8'
# track_test_ = 'spotify:track:25kXhsMguNGnSVWPxsd34Z'
# track_test_ = 'spotify:track:2ugxxlY8ukD5eINjz2Jvti'
# track_test_ = 'spotify:track:7MgkyepnUxzlvO4z65athy'
# track_test_ = 'spotify:track:5Qo2qbhJZEWE4Go6RNC4Ee'
# track_test_ = 'spotify:track:2KkqvaYudEQMQXRK8c8VTs'
# track_test_ = 'spotify:track:553ZNNYDXGnBR3HGceKWNQ'
track_test_ = 'spotify:track:3Ij3BciihXLsMIlQIn0Ob2'
track_meta = sp.track(track_test_)
print(f"track name: {track_meta['name']}")
print(f"artists name: {track_meta['artists'][0]['name']}")
print(f"track preview_url: {track_meta['preview_url']}")
print(f"track spotify url: {track_meta['external_urls']['spotify']}")

track name: Apollo
artists name: tyDi
track preview_url: https://p.scdn.co/mp3-preview/c122b7d284a81e4c83232eff5474266acdac8ea8?cid=2dce494e64a74be980138668f4402b97
track spotify url: https://open.spotify.com/track/3Ij3BciihXLsMIlQIn0Ob2


## Testing trade-offs: latency vs recall

In [ ]:
# from time import time

start_time = time.time()

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed Time: {round(elapsed_time, 2)}")